In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
bread = pd.read_csv("/content/drive/MyDrive/dataminassi/assignment4/bread.csv")
# bread

#removing duplicates
bread = bread.drop_duplicates()

# splitting the data
new = bread['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [3]:
bread['Date'] = new[0]
bread['Time'] = new[1]
bread['Transaction'] = new[2]
bread['Item'] = new[3]
# bread[['Date', 'Time', 'Transaction', 'Item']].head(10)

In [4]:
transaction = pd.crosstab(index= bread['Transaction'], columns= bread['Item'])
# transaction

## removing unwanted "NONE"
transaction = transaction.drop(['NONE'], axis = 1)

In [7]:
def AprioriAlgo(data, min_support=0.04,  max_length = 4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {} 
    L = list(data.columns)
    
    # Step 2: 
    #generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = set(combinations(L,i))
        
    # Reset "L" for next ith iteration
        L =set()     
    # Step 3: 
        #iterate through each item in "c"
        for j in list(c):
            #print(j)
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            if sup > min_support:
                #print(sup, j)
                support[j] = sup
                
                # Appending frequent itemset in list "L", already reset list "L" 
                L = list(set(L) | set(j))
        
    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
    return(result)

In [8]:
## finding frequent itemset with min support = 4%
my_freq_itemset = AprioriAlgo(transaction, 0.04, 3)

#printing frequent itemsets in decending order
my_freq_itemset.sort_values(by = 'Support', ascending = False)

,Items,Support
2,"(Coffee,)",0.475081
3,"(Bread,)",0.324940
7,"(Tea,)",0.141643
5,"(Cake,)",0.103137
9,"(Bread, Coffee)",0.089393
4,"(Pastry,)",0.085510
0,"(Sandwich,)",0.071346
6,"(Medialuna,)",0.061379
8,"(Hot chocolate,)",0.057916
12,"(Cake, Coffee)",0.054349


In [13]:
def AssociationRule(df, min_threshold=0.5):
    import pandas as pd
    from itertools import permutations
    
    #creating required varaible
    support = pd.Series(df.Support.values, index=df.Items).to_dict()
    data = []
    L= df.Items.values
    
    #generating rule using permutation
    p = list(permutations(L, 2))
    
    # Iterating through each rule
    for i in p:
        
        # If LHS(Antecedent) of rule is subset of RHS then valid rule.
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            #print(i, conf)
            if conf > min_threshold:
                #print(i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

         
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [14]:
#calling the above function on the data with threshold 0.5
my_rule = AssociationRule(my_freq_itemset, 0.5)

# filtering support values less than 0.02 from above result
newdf = my_rule[my_rule['support'] > 0.02]

#output
newdf[['antecedents','consequents','support','confidence']]

,antecedents,consequents,support,confidence
0,"(Pastry,)","(Coffee,)",0.047214,0.552147
1,"(Cake,)","(Coffee,)",0.054349,0.526958
